In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from time import process_time
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import psutil
import os
import re
import multiprocessing
import joblib
import gc
from tqdm.notebook import tqdm  # Progress bars for Jupyter

# Configuration options
VISUALIZE_OBJECTS = False  # Set to True if you want to visualize objects
SAVE_INTERMEDIATE_MODELS = True  # Set to False to save only final models

# Flag to control whether to use multiple scales or only the maximum scale
use_multiple_scales = True  # Set to False to use only maximum scale, True for all scales

# Get available CPU cores and set appropriate parallelism
n_cores = multiprocessing.cpu_count()
n_jobs = max(1, n_cores - 1)  # Leave one core free for system processes
print(f"Using {n_jobs} of {n_cores} available CPU cores")

# For reproducibility
RANDOM_SEED = 3
np.random.seed(RANDOM_SEED)

def monitor_memory():
    """Print current memory usage of the process"""
    process = psutil.Process()
    memory_mb = process.memory_info().rss / (1024 * 1024)
    print(f"Memory usage: {memory_mb:.2f} MB")

def MAPE(actual_values, predicted_values):
    """Calculate Mean Absolute Percentage Error with special handling for zeros"""
    # Vectorized implementation
    actual_flat = actual_values.flatten()
    pred_flat = predicted_values.flatten()
    
    # Create mask for non-zero actual values
    non_zero_mask = actual_flat != 0
    zero_mask = ~non_zero_mask
    
    # Calculate MAPE for non-zero elements
    mape_sum = 0
    count = len(actual_flat)
    
    if np.any(non_zero_mask):
        mape_sum += np.sum(np.abs((actual_flat[non_zero_mask] - pred_flat[non_zero_mask]) / actual_flat[non_zero_mask]))
    
    if np.any(zero_mask):
        mape_sum += np.sum(np.abs(actual_flat[zero_mask] - pred_flat[zero_mask]) / 100)
    
    return mape_sum / count

# Load spatial statistics to get universe boundaries for each dataset
print("Loading spatial statistics...")
spatial_stats = pd.read_csv('../spatial_statistics.csv')

# Directory containing the datasets
data_dir = '../large_files/resultsDistance/'  # Changed to distance folder

# Parse bounding box information
def parse_bbox(bbox_str):
    # Extract coordinates from BOX string using regex
    pattern = r"BOX\(([-\d\.]+) ([-\d\.]+),([-\d\.]+) ([-\d\.]+)\)"
    match = re.search(pattern, bbox_str)
    if match:
        xmin = float(match.group(1))
        ymin = float(match.group(2))
        xmax = float(match.group(3))
        ymax = float(match.group(4))
        return xmin, ymin, xmax, ymax
    return -180, -90, 180, 90  # Default if parsing fails

# Parse object MBR from format like "(x1, y1, x2, y2)"
def parse_mbr(mbr_str):
    coords = mbr_str.strip('"()').split(', ')
    return [float(coord) for coord in coords]

# Extract universe boundaries for each dataset
universe_boundaries = {}
for _, row in spatial_stats.iterrows():
    table_name = row['Table Name']
    bbox = parse_bbox(row['Universe Limits (Bounding Box)'])
    universe_boundaries[table_name] = bbox

# Get list of all CSV files in the directory
print("Finding dataset files...")
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
print(f"Found {len(csv_files)} datasets to process")

# Define the scales of learning
scales = [1000, 5000, 10000, 50000, 100000, 500000, 1000000]

# Create necessary directories
os.makedirs('../large_files/LearnedModels/distance/NN', exist_ok=True)
os.makedirs('../large_files/LearnedModels/distance/NN/visualizations', exist_ok=True)
os.makedirs('../large_files/LearnedModels/distance/NN/results', exist_ok=True)

# Lists to store all results
all_results_list = []

# Process each dataset
for csv_file in tqdm(csv_files, desc="Processing datasets"):
    # Force garbage collection at the start of each dataset
    gc.collect()
    monitor_memory()
    
    # Extract dataset name (remove "_results.csv")
    dataset_name = csv_file.replace('_results.csv', '')
    
    print(f"\nProcessing dataset: {dataset_name}")
    
    # Get universe boundaries for this dataset
    if dataset_name in universe_boundaries:
        univ_xmin, univ_ymin, univ_xmax, univ_ymax = universe_boundaries[dataset_name]
    else:
        # Default values if dataset not found in spatial stats
        univ_xmin, univ_ymin, univ_xmax, univ_ymax = -180, -90, 180, 90
    
    Surface_univ = (univ_xmax - univ_xmin) * (univ_ymax - univ_ymin)
    print(f"Universe boundaries for {dataset_name}: ({univ_xmin}, {univ_ymin}, {univ_xmax}, {univ_ymax})")
    
    # Load dataset - loading required columns for distance dataset
    data_path = os.path.join(data_dir, csv_file)
    print(f"Loading data from {data_path}")
    data = pd.read_csv(data_path, usecols=['Object MBR', 'Distance Min', 'Distance Max', 'Count MBR'])
    
    # Extract object information - ONLY using MBR data
    print("Parsing object coordinates...")
    Objects_MBR = np.array([parse_mbr(mbr) for mbr in data['Object MBR']])
    
    # Extract distance information
    Distance_Min = data['Distance Min'].values.reshape(-1, 1)
    Distance_Max = data['Distance Max'].values.reshape(-1, 1)
    
    # Target variable: Count MBR
    Y = data[['Count MBR']].values
    
    # Free up memory
    del data
    gc.collect()
    
    # Calculate basic statistics
    max_count = float(np.max(Y))
    min_count = float(np.min(Y))
    mean_count = float(np.mean(Y))
    median_count = float(np.median(Y))
    total_samples = len(Y)

    # Display basic statistics for the dataset
    print(f"\nBasic statistics for {dataset_name} dataset:")
    print(f"Max count: {max_count}")
    print(f"Min count: {min_count}")
    print(f"Mean count: {mean_count:.2f}")
    print(f"Median count: {median_count:.2f}")
    print(f"Total samples: {total_samples}\n")

    # Prepare features using Object MBR instead of True Shape
    print("Calculating object features...")

    # Extract MBR coordinates
    x1 = Objects_MBR[:, 0].reshape(-1, 1)  # Left
    y1 = Objects_MBR[:, 1].reshape(-1, 1)  # Bottom
    x2 = Objects_MBR[:, 2].reshape(-1, 1)  # Right
    y2 = Objects_MBR[:, 3].reshape(-1, 1)  # Top

    # Calculate MBR center points
    obj_x = (x1 + x2) / 2  # Center X
    obj_y = (y1 + y2) / 2  # Center Y

    # Calculate MBR dimensions
    mbr_width = (x2 - x1)
    mbr_height = (y2 - y1)
    mbr_area = mbr_width * mbr_height

    # Normalized coordinates of MBR center (0-1 range within universe)
    norm_x = (obj_x - univ_xmin) / (univ_xmax - univ_xmin) if (univ_xmax - univ_xmin) != 0 else 0.5
    norm_y = (obj_y - univ_ymin) / (univ_ymax - univ_ymin) if (univ_ymax - univ_ymin) != 0 else 0.5

    # Distance range
    distance_range = Distance_Max - Distance_Min

    # Distance ratio (max/min)
    # Avoid division by zero
    min_non_zero = np.where(Distance_Min == 0, 0.0001, Distance_Min)
    distance_ratio = Distance_Max / min_non_zero
    
    # Combine all features
    X = np.hstack((
        obj_x,           # X coordinate of MBR center
        obj_y,           # Y coordinate of MBR center
        mbr_width,       # Width of MBR
        mbr_height,      # Height of MBR
        mbr_area,        # Area of MBR
        norm_x,          # Normalized X position (0-1)
        norm_y,          # Normalized Y position (0-1)
        Distance_Min,    # Minimum distance
        Distance_Max,    # Maximum distance
        distance_range,  # Range of distance
        distance_ratio.reshape(-1, 1)  # Ratio of max/min distance
    ))
    
    # Split the data into 80% train and 20% test
    print("Splitting data into train and test sets...")
    X_train, X_test_all, y_train, y_test_all = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
    
    # Visualize the first 100 objects using MBRs
    if VISUALIZE_OBJECTS:
        print("Visualizing objects sample...")
        plt.figure(figsize=(10, 8))
        
        # Only visualize a sample to save time
        sample_size = min(100, len(Objects_MBR))
        
        # Plot universe boundaries
        plt.plot([univ_xmin, univ_xmax, univ_xmax, univ_xmin, univ_xmin], 
                 [univ_ymin, univ_ymin, univ_ymax, univ_ymax, univ_ymin], 
                 'k-', linewidth=1, alpha=0.5)
        
        # Plot MBRs
        for i in range(sample_size):
            x1, y1, x2, y2 = Objects_MBR[i]
            rect = patches.Rectangle((x1, y1), x2-x1, y2-y1, 
                                    linewidth=1, edgecolor='b', facecolor='none', 
                                    alpha=min(1.0, 0.3 + float(Distance_Min[i])/20))
            plt.gca().add_patch(rect)
            
        plt.colorbar(plt.cm.ScalarMappable(cmap='viridis'), label='Minimum Distance')
        plt.xlim(univ_xmin-20, univ_xmax+20)
        plt.ylim(univ_ymin-10, univ_ymax+10)
        plt.title(f"Sample MBRs from {dataset_name}")
        plt.savefig(f"../large_files/LearnedModels/distance/NN/visualizations/{dataset_name}_objects_mbr.png", dpi=150)
        plt.close()
    
    # Adjust scales to the dataset size
    max_size = len(X_train)
    print(f"Training set size: {max_size}")

    if use_multiple_scales:
        # Use multiple scales
        adjusted_scales = [s for s in scales if s <= max_size]

        # Add intermediate 1 million increments for large datasets
        if max_size > 1000000:
            million_increments = list(range(2000000, max_size, 1000000))
            adjusted_scales.extend(million_increments)
        
        # Add the actual max size if it's not already in the list
        if max_size not in adjusted_scales:
            adjusted_scales.append(max_size)
            
        # Sort the scales to ensure they're in ascending order (smallest first)
        adjusted_scales.sort()
    else:
        # Use only the maximum scale
        adjusted_scales = [max_size]

    # List to store dataset-specific results
    dataset_results_list = []
    
    # Create a scaler for normalizing inputs
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train[:max_size])
    X_test_scaled = scaler.transform(X_test_all)
    
    # Save the scaler for future use
    joblib.dump(scaler, f'../large_files/LearnedModels/distance/NN/{dataset_name}_scaler.joblib')
    
    # Variables to track the model across scales
    cumulative_training_time = 0
    current_model = None
    previous_sample_size = 0
    
    # Process scales in ascending order (smallest first)
    for i, sample_size in enumerate(adjusted_scales):
        print(f"\nTraining with sample size: {sample_size}")
        monitor_memory()
        
        # Create training subset for this scale
        X_train_sample = X_train_scaled[:sample_size, :]
        y_train_sample = y_train[:sample_size].ravel()
        
        # Determine if we should create a new model or continue training
        create_new_model = (
            current_model is None or                         # First iteration
            sample_size >= 5 * previous_sample_size or       # Significant increase in data
            (i > 0 and sample_size / adjusted_scales[i-1] > 3)  # More than 3x previous scale
        )
        
        # Set network architecture based on dataset size
        if sample_size < 5000:
            hidden_layer_sizes = (32, 16)
        elif sample_size < 50000:
            hidden_layer_sizes = (64, 32, 16)
        else:
            hidden_layer_sizes = (128, 64, 32, 16)
            
        # Time the training
        t_start = process_time()
        
        if create_new_model:
            print(f"Creating new neural network with architecture: {hidden_layer_sizes}")
            
            # Create a new model
            current_model = MLPRegressor(
                hidden_layer_sizes=hidden_layer_sizes,
                activation='relu',
                solver='adam',
                alpha=0.0001,
                batch_size=min(32, sample_size),
                learning_rate_init=0.001,
                max_iter=100,
                early_stopping=True,
                validation_fraction=0.1,
                n_iter_no_change=10,
                random_state=RANDOM_SEED,
                verbose=True,
                warm_start=False  # No warm start for new model
            )
        else:
            print(f"Continuing training with warm start from previous scale")
            # Adjust hyperparameters for continued training
            current_model.batch_size = min(32, sample_size)
            current_model.warm_start = True
            
            # Optionally adjust learning rate for fine-tuning
            if sample_size >= 200000:
                current_model.learning_rate_init = 0.0005
        
        # Train the model
        current_model.fit(X_train_sample, y_train_sample)
        
        t_stop = process_time()
        training_time = t_stop - t_start
        cumulative_training_time += training_time
        
        # Update previous sample size for next iteration
        previous_sample_size = sample_size
        
        # Make predictions
        print("Making predictions...")
        y_pred = current_model.predict(X_test_scaled).reshape(-1, 1)
        # Ensure non-negative predictions
        y_pred = np.maximum(0, y_pred)

        # Calculate metrics
        r2_score = current_model.score(X_test_scaled, y_test_all.ravel())
        mae_value = MAE(y_test_all, y_pred)
        mape_value = MAPE(y_test_all, y_pred)
        
        # Calculate q-score - vectorized version
        print("Calculating performance metrics...")
        
        # Vectorized q-score calculation
        y_true_flat = y_test_all.flatten()
        y_pred_flat = y_pred.flatten() if y_pred.ndim > 1 else y_pred
        
        # Find indices where both values are non-zero
        valid_indices = (y_true_flat != 0) & (y_pred_flat != 0)
        
        if np.any(valid_indices):
            ratios = np.maximum(
                y_pred_flat[valid_indices] / y_true_flat[valid_indices],
                y_true_flat[valid_indices] / y_pred_flat[valid_indices]
            )
            q_score_mean = np.mean(ratios)
        else:
            q_score_mean = 0
        
        # Time prediction performance (10 iterations)
        print("Measuring prediction performance...")
        total_duration = 0
        total_read = 0
        total_write = 0
        
        for _ in range(10):
            io_before = psutil.disk_io_counters()
            t3_start = process_time()
            preds = current_model.predict(X_test_scaled)
            preds = np.maximum(0, preds)  # Ensure non-negative predictions
            t3_stop = process_time()
            io_after = psutil.disk_io_counters()
            
            total_duration += (t3_stop - t3_start)
            total_read += io_after.read_count - io_before.read_count
            total_write += io_after.write_count - io_before.write_count
        
        avg_pred_time_microsec = (total_duration / 10) / len(y_pred) * 1000000
        avg_reads = total_read / 10 / len(y_pred)
        avg_writes = total_write / 10 / len(y_pred)
        
        # Save the model using joblib with incremental suffix
        if SAVE_INTERMEDIATE_MODELS or sample_size == max_size:
            print("Saving model...")
            model_path = f'../large_files/LearnedModels/distance/NN/{dataset_name}_nn_{sample_size}_cumul_{cumulative_training_time:.2f}s_{mape_value:.2%}_{mae_value:.2f}.joblib'
            joblib.dump(current_model, model_path, compress=3)

            # Get model file size in KB
            model_size_kb = os.path.getsize(model_path) / 1024
            print(f"Model size: {model_size_kb:.2f} KB")
        else:
            model_size_kb = 0  # Set to 0 if model wasn't saved
        
        # Plot learning curve for maximum scale
        if sample_size == max_size:
            plt.figure(figsize=(12, 6))
            plt.plot(current_model.loss_curve_, label='Training Loss')
            plt.title('MLPRegressor Loss Curve')
            plt.xlabel('Iterations')
            plt.ylabel('Loss')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.savefig(f"../large_files/LearnedModels/distance/NN/visualizations/{dataset_name}_loss_curve.png", dpi=150)
            plt.close()
        
        # Print results
        print(f"\nResults for {dataset_name}, Sample Size: {sample_size}")
        print(f"Training Time (this scale): {training_time:.2f}s")
        print(f"Cumulative Training Time: {cumulative_training_time:.2f}s")
        print(f"Neural Network Architecture: {hidden_layer_sizes}")
        print(f"Performance: R² = {r2_score:.4f}, MAE = {mae_value:.2f}, MAPE = {mape_value:.2%}")
        print(f"q-score: {q_score_mean:.2f}")
        print(f"Prediction time: {avg_pred_time_microsec:.4f} μs/sample")
        print(f"I/O: Reads={avg_reads:.6f}, Writes={avg_writes:.6f}")
        print("-" * 80)
        
        # Plot actual vs predicted only for the maximum scale
        if sample_size == adjusted_scales[-1]:
            print("Generating prediction scatter plot...")
            plt.figure(figsize=(10, 8))
            plt.scatter(y_test_all, y_pred, s=0.5, alpha=0.5)
            plt.xlabel('True Values')
            plt.ylabel('Predictions')
            plt.title(f"{dataset_name} - Sample Size: {sample_size} (Maximum)")
            plt.grid(True, alpha=0.3)
            
            # Add diagonal line for perfect predictions
            max_val = max(np.max(y_test_all), np.max(y_pred))
            plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.5)
            
            plt.savefig(f"../large_files/LearnedModels/distance/NN/visualizations/{dataset_name}_{sample_size}_prediction.png", dpi=150)
            plt.close()
            
            # Create sample comparison plot
            print("Generating side-by-side comparison plot...")
            
            # Get predictions for first 100 test samples
            sample_indices = range(min(100, len(X_test_scaled)))
            X_sample = X_test_scaled[sample_indices]
            y_sample_true = y_test_all[sample_indices].flatten()
            y_sample_pred = current_model.predict(X_sample)
            # Ensure non-negative predictions
            y_sample_pred = np.maximum(0, y_sample_pred)
            
            plt.figure(figsize=(20, 10))
            plt.scatter(range(len(sample_indices)), y_sample_pred, c='orange', 
                        label='Predicted number of objects (Neural Network)', alpha=0.7, s=100)
            plt.scatter(range(len(sample_indices)), y_sample_true, c='green', 
                        label='Real number of objects', alpha=0.7, s=100)
            
            plt.title(f'{dataset_name} - First {len(sample_indices)} Objects: Predicted vs Real Values', fontsize=16)
            plt.xlabel('Object Index', fontsize=14)
            plt.ylabel('Number of objects within distance', fontsize=14)
            plt.legend(fontsize=12)
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            
            # Save the plot
            plt.savefig(f"../large_files/LearnedModels/distance/NN/visualizations/{dataset_name}_comparison_plot.png", dpi=150)
            plt.close()
        
        # Store results in list with cumulative training time
        result_row = {
            'Dataset': dataset_name,
            'Sample_Size': sample_size,
            'Training_Time_This_Scale': training_time,
            'Cumulative_Training_Time': cumulative_training_time,
            'NN_Architecture': str(hidden_layer_sizes),
            'R2_Score': r2_score,
            'MAE': mae_value,
            'MAPE': float(mape_value),
            'Q_Score': q_score_mean,
            'Pred_Time_Microseconds': avg_pred_time_microsec,
            'IO_Reads': avg_reads,
            'IO_Writes': avg_writes,
            'Model_Size_KB': model_size_kb,
            'Max_Count': max_count,
            'Min_Count': min_count,
            'Mean_Count': mean_count,
            'Median_Count': median_count,
            'Total_Samples': total_samples,
            'Warm_Start': not create_new_model
        }
        
        dataset_results_list.append(result_row)
        all_results_list.append(result_row)
    
    # Save results for this dataset
    print(f"Saving results for {dataset_name}...")
    dataset_results = pd.DataFrame(dataset_results_list)
    dataset_results.to_csv(f'../large_files/LearnedModels/distance/NN/results/{dataset_name}_results.csv', index=False)
    
    # Clear memory before next dataset
    del X_train, X_test_all, y_train, y_test_all, X_train_scaled, X_test_scaled
    del Objects_MBR, Distance_Min, Distance_Max, current_model
    gc.collect()
    
# Save all results
print("Saving combined results...")
all_results = pd.DataFrame(all_results_list)
all_results.to_csv('../large_files/LearnedModels/distance/NN/all_results.csv', index=False)

print("All processing completed and results saved.")
monitor_memory()

Using 29 of 30 available CPU cores
Loading spatial statistics...
Finding dataset files...
Found 14 datasets to process


Processing datasets:   0%|          | 0/14 [00:00<?, ?it/s]

Memory usage: 235.77 MB

Processing dataset: craftwaysorted
Universe boundaries for craftwaysorted: (-175.2000514, -65.2458821, 175.3397782, 69.6673353)
Loading data from ../large_files/resultsDistance/craftwaysorted_results.csv
Parsing object coordinates...



Basic statistics for craftwaysorted dataset:
Max count: 100369.0
Min count: 0.0
Mean count: 11615.73
Median count: 3842.00
Total samples: 21822

Calculating object features...
Splitting data into train and test sets...
Training set size: 17457

Training with sample size: 1000
Memory usage: 246.87 MB
Creating new neural network with architecture: (32, 16)


Iteration 1, loss = 253872717.22113118
Validation score: -0.440271
Iteration 2, loss = 253863762.05878729
Validation score: -0.440227
Iteration 3, loss = 253852369.03272402
Validation score: -0.440165
Iteration 4, loss = 253836806.34764928
Validation score: -0.440077
Iteration 5, loss = 253813291.54934004
Validation score: -0.439941
Iteration 6, loss = 253775604.06108594
Validation score: -0.439742
Iteration 7, loss = 253723473.46115202
Validation score: -0.439458
Iteration 8, loss = 253646292.66711968
Validation score: -0.439043
Iteration 9, loss = 253535747.39917347
Validation score: -0.438484
Iteration 10, loss = 253384912.50493321
Validation score: -0.437743
Iteration 11, loss = 253195677.19298497
Validation score: -0.436723
Iteration 12, loss = 252940965.64407757
Validation score: -0.435471
Iteration 13, loss = 252620611.78872812
Validation score: -0.433912
Iteration 14, loss = 252227394.82029423
Validation score: -0.432009
Iteration 15, loss = 251751827.37113950
Validation score:

...

Iteration 62, loss = 1515239.35957636
Validation score: 0.994206


Iteration 63, loss = 1509837.73277771
Validation score: 0.993736


Iteration 64, loss = 1520371.39869397
Validation score: 0.994352
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Making predictions...
Calculating performance metrics...
Measuring prediction performance...
Saving model...
Model size: 82.18 KB



Results for arealm, Sample Size: 20666
Training Time (this scale): 946.24s
Cumulative Training Time: 1875.00s
Neural Network Architecture: (64, 32, 16)
Performance: R² = 0.9929, MAE = 1313.47, MAPE = 83.29%
q-score: 1.84
Prediction time: 23.0086 μs/sample
I/O: Reads=0.000000, Writes=0.000000
--------------------------------------------------------------------------------
Generating prediction scatter plot...
Generating side-by-side comparison plot...


Saving results for arealm...
Saving combined results...
All processing completed and results saved.
Memory usage: 836.66 MB
